# Мастерская. Проект по предсказанию сердечного приступа.

В данном проекте нам предстоит нам предстоит разработать модель машинного обучния, а также подготовить библиотеку и интерфейс к ней для предсказания на тестовой выборке. 

В начале проекта нам доступен обучающий датасет и тестовый датасет, не содержащий правильных ответов. Метрика на тестовом датасете будет измерена в ходе проверки работы. Также нам доступен скрипт, с помощью которого будет измеряться финальная метрика вашей модели.

В данной работе мы сделаем следующие шаги:
- изучим предоставленные данные,
- проведем предобработку,
- исследования,
- объединение данных и выделение необходимых признаков,
- изучим взаимосвязь параметров между собой,
- построим модели,
- подобрем гипермараметры,
- выберем наиболее подходящую из них,
- подберем наиболее подходящую для проекта метрику,
- сделаем предсказание для тестовой выборки для отправки на проверку,
- кроме того, мы подготовим библиотеку и интерфейс к ней.

In [1]:
#импорты и константы

import numpy as np
import pandas as pd
!pip install --upgrade matplotlib -q
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as st
!pip install --upgrade scikit-learn -q
from sklearn.model_selection import train_test_split

# загружаем класс pipeline
from sklearn.pipeline import Pipeline

# загружаем классы для подготовки данных
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder, 
    StandardScaler, 
    MinMaxScaler,
    LabelEncoder
)
from sklearn.compose import ColumnTransformer

# загружаем класс для работы с пропусками
from sklearn.impute import SimpleImputer

# загружаем функцию для работы с метриками
from sklearn.metrics import roc_auc_score, make_scorer

# импортируем класс RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# загружаем нужные модели
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.svm import SVC

!pip install phik -q
from phik import phik_matrix
from phik.report import plot_correlation_matrix


!pip install shap -q
import shap

RANDOM_STATE = 42
TEST_SIZE = 0.25

import warnings
#warnings.filterwarnings('ignore', category=FutureWarning)

## Загрузка данных

В начале работы загрузим и изучим данные.

In [2]:
# функция для изучения данных
def describe_db(name):
    print(name.info())
    display(name.shape)
    display(name.head())
    print('Дубликаты:', name.duplicated().sum())
    print()
    print('Пропуски:')
    print(name.isna().sum())
    print()

In [3]:
train_X_y = pd.read_csv('./heart_train.csv')
describe_db(train_X_y)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8685 entries, 0 to 8684
Data columns (total 28 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       8685 non-null   int64  
 1   Age                              8685 non-null   float64
 2   Cholesterol                      8685 non-null   float64
 3   Heart rate                       8685 non-null   float64
 4   Diabetes                         8442 non-null   float64
 5   Family History                   8442 non-null   float64
 6   Smoking                          8442 non-null   float64
 7   Obesity                          8442 non-null   float64
 8   Alcohol Consumption              8442 non-null   float64
 9   Exercise Hours Per Week          8685 non-null   float64
 10  Diet                             8685 non-null   int64  
 11  Previous Heart Problems          8442 non-null   float64
 12  Medication Use      

(8685, 28)

,Unnamed: 0,Age,Cholesterol,Heart rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,...,Physical Activity Days Per Week,Sleep Hours Per Day,Heart Attack Risk (Binary),Blood sugar,CK-MB,Troponin,Gender,Systolic blood pressure,Diastolic blood pressure,id
0,0,0.359551,0.732143,0.074244,1.0,1.0,1.0,1.0,1.0,0.535505,...,3.0,0.333333,0.0,0.227018,0.048229,0.036512,Male,0.212903,0.709302,2664
1,1,0.202247,0.325000,0.047663,1.0,1.0,0.0,0.0,1.0,0.068690,...,3.0,0.833333,0.0,0.150198,0.017616,0.000194,Female,0.412903,0.569767,9287
2,2,0.606742,0.860714,0.055912,1.0,0.0,1.0,1.0,1.0,0.944001,...,2.0,1.000000,0.0,0.227018,0.048229,0.036512,Female,0.238710,0.220930,5379
3,3,0.730337,0.007143,0.053162,0.0,0.0,1.0,0.0,1.0,0.697023,...,0.0,0.333333,1.0,0.227018,0.048229,0.036512,Female,0.348387,0.267442,8222
4,4,0.775281,0.757143,0.021998,0.0,0.0,1.0,0.0,1.0,0.412878,...,5.0,1.000000,1.0,0.227018,0.048229,0.036512,Male,0.619355,0.441860,4047


Дубликаты: 0

Пропуски:
Unnamed: 0                           0
Age                                  0
Cholesterol                          0
Heart rate                           0
Diabetes                           243
Family History                     243
Smoking                            243
Obesity                            243
Alcohol Consumption                243
Exercise Hours Per Week              0
Diet                                 0
Previous Heart Problems            243
Medication Use                     243
Stress Level                       243
Sedentary Hours Per Day              0
Income                               0
BMI                                  0
Triglycerides                        0
Physical Activity Days Per Week    243
Sleep Hours Per Day                  0
Heart Attack Risk (Binary)           0
Blood sugar                          0
CK-MB                                0
Troponin                             0
Gender                               0
S

In [4]:
test_X_y = pd.read_csv('./heart_test.csv')
describe_db(test_X_y)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966 entries, 0 to 965
Data columns (total 27 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       966 non-null    int64  
 1   Age                              966 non-null    float64
 2   Cholesterol                      966 non-null    float64
 3   Heart rate                       966 non-null    float64
 4   Diabetes                         935 non-null    float64
 5   Family History                   935 non-null    float64
 6   Smoking                          935 non-null    float64
 7   Obesity                          935 non-null    float64
 8   Alcohol Consumption              935 non-null    float64
 9   Exercise Hours Per Week          966 non-null    float64
 10  Diet                             966 non-null    int64  
 11  Previous Heart Problems          935 non-null    float64
 12  Medication Use        

(966, 27)

,Unnamed: 0,Age,Cholesterol,Heart rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,...,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Blood sugar,CK-MB,Troponin,Gender,Systolic blood pressure,Diastolic blood pressure,id
0,0,0.494382,0.264286,0.062328,0.0,1.0,1.0,1.0,1.0,0.361618,...,0.306494,1.0,0.333333,0.227018,0.048229,0.036512,Male,0.283871,0.372093,7746
1,1,0.224719,0.953571,0.082493,1.0,0.0,0.0,1.0,0.0,0.996483,...,0.087013,0.0,0.166667,0.227018,0.048229,0.036512,Female,0.703226,0.441860,4202
2,2,0.629213,0.092857,0.064161,0.0,1.0,1.0,1.0,0.0,0.995561,...,0.205195,7.0,1.000000,0.102767,0.002666,0.088455,Male,0.458065,0.779070,6632
3,3,0.460674,0.567857,0.055912,1.0,1.0,1.0,1.0,1.0,0.437277,...,0.163636,0.0,0.666667,0.203557,0.056390,0.271774,Female,0.741935,0.255814,4639
4,4,0.719101,0.485714,0.022915,1.0,0.0,1.0,0.0,1.0,0.514920,...,0.580519,5.0,0.000000,0.227018,0.048229,0.036512,Male,0.412903,0.395349,4825


Дубликаты: 0

Пропуски:
Unnamed: 0                          0
Age                                 0
Cholesterol                         0
Heart rate                          0
Diabetes                           31
Family History                     31
Smoking                            31
Obesity                            31
Alcohol Consumption                31
Exercise Hours Per Week             0
Diet                                0
Previous Heart Problems            31
Medication Use                     31
Stress Level                       31
Sedentary Hours Per Day             0
Income                              0
BMI                                 0
Triglycerides                       0
Physical Activity Days Per Week    31
Sleep Hours Per Day                 0
Blood sugar                         0
CK-MB                               0
Troponin                            0
Gender                              0
Systolic blood pressure             0
Diastolic blood pressure  

Самое заметное в данных - они масштабированы. Скорее всего - в целях конфиденциальности. Так же скрыты и личные данные пациентов.

В данных тренировочных и тестовых наблюдаются колонки с неверными типами данных (float для целочисленных, скорее всего, колонок). Полных дубликатов не наблюдается, есть пропуски, в одинаковом количестве для разных столбцов. Предполагаю, что они возникли из-за отсутствия данных, а не из-за ошибок. Данных получено достаточно много: почти 9000 на обучение и почти 1000 значений в тестовой выборке. Предполагаю, что этого должно хватить на качественное обучение модели.

В целом, присутствует очень много колонок с различными характеристиками. В дальнейшем вникнем в их содержание, поправим форматы, примем решение по пропускам, исключим колонки, которые будут неверно влиять на обучение (скорее всего, под такое исключение попадет колонка с номером пациента, id и возможные утечки данных). Править мы будем только тренировочные данные, тестовые останутся без изменений.

## Предобработка данных

Как уже выяснили выше, полных дубликатов в таблицах не обнаружено. Попробуем проверить, что будет, если исключить столбцы Unnamed и id из таблиц. Для начала проверим количество уникальных значений в этих столбцах.

In [5]:
def count_unique(column):
    print(column.name)
    print(len(column.unique()))

In [6]:
count_unique(train_X_y['Unnamed: 0'])
count_unique(train_X_y['id'])

Unnamed: 0
8685
id
8685


Видно, что нет дубликатов и пропусков в этих столбцах. Значит, эти значения уникальны и привязаны к каждому отдельному пациенту. Они могут негативно повлиять на обучение, например, если каким-то образом пациенты записаны не в случайном порядке, а сначала те, у кого был сердечный приступ, а потом те, у кого не было. Столбцы удаляем, в дальнейшем они нам не принесут пользы.

In [7]:
train_X_y = train_X_y.drop(['Unnamed: 0', 'id'], axis=1)
describe_db(train_X_y)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8685 entries, 0 to 8684
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              8685 non-null   float64
 1   Cholesterol                      8685 non-null   float64
 2   Heart rate                       8685 non-null   float64
 3   Diabetes                         8442 non-null   float64
 4   Family History                   8442 non-null   float64
 5   Smoking                          8442 non-null   float64
 6   Obesity                          8442 non-null   float64
 7   Alcohol Consumption              8442 non-null   float64
 8   Exercise Hours Per Week          8685 non-null   float64
 9   Diet                             8685 non-null   int64  
 10  Previous Heart Problems          8442 non-null   float64
 11  Medication Use                   8442 non-null   float64
 12  Stress Level        

(8685, 26)

,Age,Cholesterol,Heart rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,Diet,...,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Heart Attack Risk (Binary),Blood sugar,CK-MB,Troponin,Gender,Systolic blood pressure,Diastolic blood pressure
0,0.359551,0.732143,0.074244,1.0,1.0,1.0,1.0,1.0,0.535505,1,...,0.979221,3.0,0.333333,0.0,0.227018,0.048229,0.036512,Male,0.212903,0.709302
1,0.202247,0.325000,0.047663,1.0,1.0,0.0,0.0,1.0,0.068690,2,...,0.515584,3.0,0.833333,0.0,0.150198,0.017616,0.000194,Female,0.412903,0.569767
2,0.606742,0.860714,0.055912,1.0,0.0,1.0,1.0,1.0,0.944001,2,...,0.012987,2.0,1.000000,0.0,0.227018,0.048229,0.036512,Female,0.238710,0.220930
3,0.730337,0.007143,0.053162,0.0,0.0,1.0,0.0,1.0,0.697023,0,...,0.131169,0.0,0.333333,1.0,0.227018,0.048229,0.036512,Female,0.348387,0.267442
4,0.775281,0.757143,0.021998,0.0,0.0,1.0,0.0,1.0,0.412878,1,...,0.070130,5.0,1.000000,1.0,0.227018,0.048229,0.036512,Male,0.619355,0.441860


Дубликаты: 0

Пропуски:
Age                                  0
Cholesterol                          0
Heart rate                           0
Diabetes                           243
Family History                     243
Smoking                            243
Obesity                            243
Alcohol Consumption                243
Exercise Hours Per Week              0
Diet                                 0
Previous Heart Problems            243
Medication Use                     243
Stress Level                       243
Sedentary Hours Per Day              0
Income                               0
BMI                                  0
Triglycerides                        0
Physical Activity Days Per Week    243
Sleep Hours Per Day                  0
Heart Attack Risk (Binary)           0
Blood sugar                          0
CK-MB                                0
Troponin                             0
Gender                               0
Systolic blood pressure              0
D

Дубликатов нет даже после удаления ненужных столбцов. Теперь проверим содержимое единственного текстового столбца Gender.

In [8]:
train_X_y['Gender'].unique()

array(['Male', 'Female', '1.0', '0.0'], dtype=object)

Опечаток нет, но мужской и женский пол для некоторых значений заменены на 1 и 0, соответственно. Сделаем эти значения единообразными.

In [9]:
train_X_y['Gender'] = train_X_y['Gender'].replace({'Female': 0.0, 'Male': 1.0}).astype(float).astype(int)

train_X_y['Gender'].unique()

array([1, 0])

Заменим типы данных и для других столбцов c целыми числами, заодно заменим пропуски на значение -1. Заменять пропуски средними или наиболее частыми значениями мы не будем, так как это живые данные и не стоит выдумывать тут свои числа.

In [19]:
train_X_y.nunique()

Age                                  77
Cholesterol                         282
Heart rate                           87
Diabetes                              3
Family History                        3
Smoking                               3
Obesity                               3
Alcohol Consumption                   3
Exercise Hours Per Week            7933
Diet                                  4
Previous Heart Problems               3
Medication Use                        3
Stress Level                         11
Sedentary Hours Per Day            7933
Income                             7808
BMI                                7933
Triglycerides                       772
Physical Activity Days Per Week       9
Sleep Hours Per Day                   2
Heart Attack Risk (Binary)            2
Blood sugar                         239
CK-MB                               679
Troponin                            340
Gender                                2
Systolic blood pressure             100


In [11]:
train_X_y['Stress Level'].unique()

array([ 8.,  9.,  6.,  3.,  7.,  1., 10., nan,  2.,  5.,  4.])

In [12]:
for column in train_X_y.columns:
    if len(train_X_y[column].unique()) <= 11:
        train_X_y[column] = train_X_y[column].fillna(-1).astype('int64')

In [14]:
describe_db(train_X_y)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8685 entries, 0 to 8684
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              8685 non-null   float64
 1   Cholesterol                      8685 non-null   float64
 2   Heart rate                       8685 non-null   float64
 3   Diabetes                         8685 non-null   int64  
 4   Family History                   8685 non-null   int64  
 5   Smoking                          8685 non-null   int64  
 6   Obesity                          8685 non-null   int64  
 7   Alcohol Consumption              8685 non-null   int64  
 8   Exercise Hours Per Week          8685 non-null   float64
 9   Diet                             8685 non-null   int64  
 10  Previous Heart Problems          8685 non-null   int64  
 11  Medication Use                   8685 non-null   int64  
 12  Stress Level        

(8685, 26)

,Age,Cholesterol,Heart rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,Diet,...,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Heart Attack Risk (Binary),Blood sugar,CK-MB,Troponin,Gender,Systolic blood pressure,Diastolic blood pressure
0,0.359551,0.732143,0.074244,1,1,1,1,1,0.535505,1,...,0.979221,3,0,0,0.227018,0.048229,0.036512,1,0.212903,0.709302
1,0.202247,0.325000,0.047663,1,1,0,0,1,0.068690,2,...,0.515584,3,0,0,0.150198,0.017616,0.000194,0,0.412903,0.569767
2,0.606742,0.860714,0.055912,1,0,1,1,1,0.944001,2,...,0.012987,2,1,0,0.227018,0.048229,0.036512,0,0.238710,0.220930
3,0.730337,0.007143,0.053162,0,0,1,0,1,0.697023,0,...,0.131169,0,0,1,0.227018,0.048229,0.036512,0,0.348387,0.267442
4,0.775281,0.757143,0.021998,0,0,1,0,1,0.412878,1,...,0.070130,5,1,1,0.227018,0.048229,0.036512,1,0.619355,0.441860


Дубликаты: 0

Пропуски:
Age                                0
Cholesterol                        0
Heart rate                         0
Diabetes                           0
Family History                     0
Smoking                            0
Obesity                            0
Alcohol Consumption                0
Exercise Hours Per Week            0
Diet                               0
Previous Heart Problems            0
Medication Use                     0
Stress Level                       0
Sedentary Hours Per Day            0
Income                             0
BMI                                0
Triglycerides                      0
Physical Activity Days Per Week    0
Sleep Hours Per Day                0
Heart Attack Risk (Binary)         0
Blood sugar                        0
CK-MB                              0
Troponin                           0
Gender                             0
Systolic blood pressure            0
Diastolic blood pressure           0
dtype: int64



Таким образом, мы изменили типы данных с столбцах с целыми значениями, исправили разнообразие в формате записи значений столбца Gender, заменили пропуски на значения -1. Так же удалили столбцы, не имеющие отношение к обучению модели. После всех этих действий дубликатов не появилось. Далее, мы проведем исследовательский анализ данных и постараемся исключить значения, которые являются утечкой данных.

## Исследовательский анализ  данных

In [20]:
# Функция для категориальных данных, круговая диаграмма
def categ_pieplot(df, col, title):
    display(df.groupby(col)['id'].count())
    (df[col].value_counts().plot.pie(y='id',autopct='%1.0f%%'))
    plt.title(title);
    
# Функция для количественных данных, гистограмма
def col_hist(df, col, title, label, bins=100):
    print(df[col].describe());
    
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,6))
    fig.suptitle(f"{title}", fontsize=15)
    sns.histplot(df[col], bins = bins, ax=ax[0]);
    ax[0].set_title('Гистограмма');
    ax[0].set_xlabel(label);
    ax[0].set_ylabel('Количество сотрудников');
    
    sns.boxplot(df[col], ax=ax[1])
    ax[1].set_title('Boxplot', fontsize=15)
    ax[1].set_xlabel(None)
    plt.show()
    
# Функция для количественных данных, дискретные
def col_count(df, col, title, label):
    print(df[col].describe());
    
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,6))
    fig.suptitle(f"{title}", fontsize=15)
    sns.countplot(df[col], ax=ax[0]);
    ax[0].set_title('Диаграмма');
    ax[0].set_xlabel(label);
    ax[0].set_ylabel('Количество сотрудников');
    
    sns.boxplot(df[col], ax=ax[1])
    ax[1].set_title('Boxplot', fontsize=15)
    ax[1].set_xlabel(None)
    plt.show()